Demeaned surfaces saved at /home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_{hemi}_ {para}_ {size}_demeaned.mgh

Note that the demean is across subjects not across vertices for each subject

In [ ]:
import numpy as np
import nibabel as nib
from pathlib import Path
import os

def demean_mgh(mgh_file):
    """
    Subtract mean from mgh file data
    
    Parameters:
    -----------
    mgh_file : str
        Path to input .mgh file with shape (163842, 1, 1, 64)
    """
    try:
        # Load mgh file
        img = nib.load(mgh_file)
        data = img.get_fdata()
        
        print(f"\nProcessing: {mgh_file}")
        print(f"Original shape: {data.shape}")
        
        # Reshape to 2D: (163842, 64)
        data_2d = data.reshape(163842, 64)
        
        # Calculate mean across subjects for each vertex
        vertex_means = np.mean(data_2d, axis=1, keepdims=True)
        
        # Subtract mean
        data_demeaned = data_2d - vertex_means
        
        # Reshape back to original shape
        data_demeaned_4d = data_demeaned.reshape(163842, 1, 1, 64)
        
        # Create new image with same header
        demeaned_img = nib.MGHImage(data_demeaned_4d, img.affine, img.header)
        
        # Create output filename
        output_dir = os.path.dirname(mgh_file)
        base_name = Path(mgh_file).stem
        output_file = os.path.join(output_dir, f"{base_name}_demeaned.mgh")
        
        # Save the file
        nib.save(demeaned_img, output_file)
        
        # Print statistics
        print(f"Original mean: {np.mean(data_2d):.6f}")
        print(f"Demeaned mean: {np.mean(data_demeaned):.6f}")
        print(f"Saved to: {output_file}")
        
        return True, output_file
        
    except Exception as e:
        print(f"Error processing {mgh_file}:")
        print(f"Error message: {str(e)}")
        return False, None

def process_all_mgh_files(base_dir):
    """
    Process all group mgh files in directory
    """
    # Parameters and hemispheres
    params = ['ad', 'md', 'rd', 'ak', 'mk', 'rk', 'fa']
    hemis = ['lh', 'rh']
    kernels = ['fwhm6', 'fwhm10']
    processed = []
    failed = []
    
    # Process each combination
    for hemi in hemis:
        for param in params:
            for kernel in kernels:
                mgh_file = os.path.join(base_dir, f"group_{hemi}_{param}_{kernel}.mgh")
                
                if os.path.exists(mgh_file):
                    success, output_file = demean_mgh(mgh_file)
                    
                    if success:
                        processed.append(mgh_file)
                    else:
                        failed.append(mgh_file)
        
    # Print summary
    print("\nProcessing Summary:")
    print(f"Total files processed successfully: {len(processed)}")
    print(f"Total files failed: {len(failed)}")

# Run the script
if __name__ == "__main__":
    base_dir = "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis"
    process_all_mgh_files(base_dir)